# Import

In [203]:
import pandas as pd
from jenkspy import JenksNaturalBreaks
import matplotlib.pyplot as plt
from IPython.display import display
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import pandas as pd
from apyori import apriori
import csv
from collections import Counter

In [204]:
path='/home/o/oananbeh/notebook/Experment2/src/TabTransformer-multiclass_discharge/'

In [205]:
df_main=pd.read_csv('/home/o/oananbeh/notebook/Covid_19_SA_Paper/data/df_main.csv')#,usecols = ['G.Patient','G.Gender','PO.OUTCOME']
df_lab=pd.read_csv('/home/o/oananbeh/notebook/Covid_19_SA_Paper/data/df_lab.csv')
df_xray=pd.read_csv('/home/o/oananbeh/notebook/Covid_19_SA_Paper/data/df_xray.csv')

In [206]:
# spisific visits for Lab and Xray data
df_lab=df_lab[(df_lab['visits']==1)] 
df_xray=df_xray[(df_xray['visits']==1)] 

In [207]:
# f=[]
# for x in df_xray.columns:
#     f.append(x)

# print(f)

In [208]:
#print the featrues that have 50% or more zero values 
features=df_lab.columns.tolist()
for featureName in features[4:-1]:
    numberOfZero=df_lab[featureName][(df_lab[featureName] == 0)].astype(int).count()
    total= df_lab[featureName].count()
    p=(50*total)/100 
    if (numberOfZero>p):
        print(featureName,'Total Number of Zero: ', numberOfZero,'out of', df_lab[featureName].count())

AST Total Number of Zero:  269 out of 327
CPK Total Number of Zero:  274 out of 327
Fibrinogen Total Number of Zero:  257 out of 327


In [209]:
df_lab = df_lab.drop(['INR','AST','CPK','Fibrinogen'], axis=1)

# Prepare Data

In [210]:
# Create table for missing data analysis
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [211]:
df_main.rename(columns={'PEFF.Glasgow Score': 'PEFF_Glasgow'}, inplace=True)


In [212]:
df_main.rename(columns={'G.Patient': 'PatientId'}, inplace=True)
df_lab['Patient Record'].fillna(method='ffill', inplace=True)
df_lab.rename(columns={'Patient Record': 'PatientId'}, inplace=True)
#df_lab = df_lab.drop_duplicates(subset='PatientId', keep="last")
#df_xray
df_xray['Patient Record'].fillna(method='ffill', inplace=True)
df_xray.rename(columns={'Patient Record': 'PatientId'}, inplace=True)
main_Xray_df=pd.merge(df_lab, df_xray, on='PatientId')
main_lab_df=pd.merge(main_Xray_df, df_main, on='PatientId')
#main_lab_df['quantile']=main_lab_df['PatientId']
#main_lab_df['quantile'] = pd.qcut(main_lab_df['WBC'], q=4, labels=['Group1', 'Group2', 'Group3', 'Group4'])
# main_lab_df.to_csv('output/main_lab_Xray_df.csv')


In [213]:
main_lab_df.shape

(311, 127)

In [214]:
died=main_lab_df['PO.OUTCOME'].value_counts()['died']
discharge=main_lab_df['PO.OUTCOME'].value_counts()['discharge']
print(f"Discharge: {discharge}")
print(f"Died: {died}")

Discharge: 247
Died: 61


In [215]:
main_lab_df.rename(columns={'G.Gender':'Gender','G.AGE': 'AGE','ABGF.Ph':'ABGF_Ph',
'ABGF.Pa O2':'ABGF_Pa_O2','ABGF.Pa CO2':'ABGF_Pa_CO2','ABGF.HCO3-':'ABGF_HCO3',
'PEFF.Temperature':'PEFF_Temperature','PEFF.Respiratory Rate':'PEFF_Respiratory_Rate',
'PEFF.Pulse':'PEFF_Pulse','PEFF.BP_Systolic':'PEFF_BP_Systolic','PEFF.BP_Diastolic':'PEFF_BP_Diastolic',
'PEFF_Glasgow Score':'PEFF_Glasgow','D-DIMER':'D_DIMER','Pro-BNP':'Pro_BNP','Vitamin D':'Vitamin_D',
'C.Diabetes':'C_Diabetes','C.HTN':'C_HTN','CSA.Fever':'CSA_Fever','CSA.SOB':'CSA_SOB',
'CSA.Cough':'CSA_Cough','MPA.Antibiotics':'MPA_Antibiotics','MPA.Favipiravir':'MPA_Favipiravir',
'MPA.Dexamethasone':'MPA_Dexamethasone','MPA.Convalescent plasma transfusion':'MPA_Convalescent_plasma_transfusion',
'MPA.Clexan or Heparine':'MPA_Clexan_or_Heparine','MPA.Tocilizumab':'MPA_Tocilizumab'}, inplace=True)

In [216]:
#Find Out How many days the Patient who is discharge Stay in Hospital
main_lab_df[['G.Date of Hospital Discharge','G.Date of Admission','G.Date of Death']] = main_lab_df[['G.Date of Hospital Discharge','G.Date of Admission','G.Date of Death']].apply(pd.to_datetime, format="%d-%m-%Y") 
main_lab_df['days_in_Hospital'] = (main_lab_df['G.Date of Hospital Discharge']-main_lab_df['G.Date of Admission'])

In [217]:
#Find Out How many days the Patient who is died Stay in Hospital
main_lab_df['days_in_Hospital'].fillna(( main_lab_df['G.Date of Death']-main_lab_df['G.Date of Admission']), inplace=True)

In [218]:
#Exclude FALSE and tracheotomy
main_lab_df['PO.OUTCOME']=main_lab_df[['PO.OUTCOME']][(main_lab_df['PO.OUTCOME']=="discharge") | (main_lab_df['PO.OUTCOME']=="died" )]

In [219]:
#Exclude nigative Values
main_lab_df['days_in_Hospital']=main_lab_df[['days_in_Hospital']][(main_lab_df['days_in_Hospital'].astype(int)>=0)]

In [220]:
main_lab_df['days_in_Hospital']=main_lab_df['days_in_Hospital'].replace('days',' ') 

In [221]:
main_lab_df[['PO.OUTCOME','PatientId']].groupby('PO.OUTCOME').count()

,PatientId
PO.OUTCOME,
died,61
discharge,247


In [222]:
main_lab_df[['PO.OUTCOME','PatientId','visits_y']].groupby(['visits_y','PO.OUTCOME']).count()

PatientId
visits_y PO.OUTCOME           
1        died               61
         discharge         247

In [223]:
def getJenks(df_process,columns,breaks):
    feature_df=pd.DataFrame(columns=[columns+'Group','female','male'])
   
    for indexBreaks in range(len(breaks)):
        if indexBreaks ==len(breaks)-1:
            break
        if indexBreaks==0:
           groubRangeLessThan='Less Than', breaks[indexBreaks]
           lessThan=df_process[(df_process[columns]<(breaks[indexBreaks]))]
           feature_df.loc[len(feature_df.index)] = [groubRangeLessThan, lessThan['G.Gender'].tolist().count('male'), lessThan['G.Gender'].tolist().count('female')] 


       # name='Group'+str(indexBreaks)
        groubRange=breaks[indexBreaks], breaks[indexBreaks+1]
        name=df_process[(df_process[columns].between(breaks[indexBreaks], breaks[indexBreaks+1]))]
        feature_df.loc[len(feature_df.index)] = [groubRange, name['G.Gender'].tolist().count('male'), name['G.Gender'].tolist().count('female')] 
        
        if indexBreaks==3:
            groubRangeGreaterThan= 'Greater Than',breaks[indexBreaks+1]
            greaterThan=df_process[(df_process[columns]>(breaks[indexBreaks+1]))]
            feature_df.loc[len(feature_df.index)] = [groubRangeGreaterThan, greaterThan['G.Gender'].tolist().count('male'), greaterThan['G.Gender'].tolist().count('female')] 

    
    return feature_df


In [224]:
def convertValues(df_main_new,df_main_new2,columns,breaks):
    for indexBreaks in range(len(breaks)):

        if indexBreaks ==len(breaks)-1:
            break
        first=(breaks[indexBreaks]) #first value in range 
        secound=(breaks[indexBreaks+1]) # secound value in Range
        df_main_new[columns]=df_main_new[columns].replace(to_replace ='FALSE', value =600000)
        df_main_new[columns]= pd.to_numeric(df_main_new[columns], downcast="float")
        #groups(Range)name
        newvlue=columns+':'+str(first)+'_'+str(secound)
        othervlue=columns+':< :'+str(breaks[0])
        othervlue2=columns +':> :'+str(breaks[-1])
        #loop in Rows and add value into proper list if it's in the range
        for index, row in df_main_new.iterrows():
            if (row[columns]>=first and row[columns] <=secound):
                df_main_new2[[columns]]=df_main_new2[[columns]].replace(row[columns], newvlue)    
            # else:
            #     if(row[columns]==600000):
            #         df_main_new2[[columns]]=df_main_new2[[columns]].replace(row[columns], 600000)
            else:
                if (row[columns]>breaks[-1] and row[columns]!= 600000):
                    df_main_new2[[columns]]=df_main_new2[[columns]].replace(row[columns], othervlue2)
                else:
                    if (row[columns]<breaks[0] and row[columns]!= 600000 ):
                        df_main_new2[[columns]]=df_main_new2[[columns]].replace(row[columns], othervlue)
    return df_main_new2[ ['AGE','ABGF_Ph','ABGF_Pa_O2', 'ABGF_Pa_CO2', 'ABGF_HCO3','PEFF_Temperature', 'PEFF_Respiratory_Rate','PEFF_Pulse', 'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow','WBC','PNN','Lymphocytes','Hemoglobin','Platelets','Creatinine','ALT', 'LDH','FERRITIN','D_DIMER','CRP','PROCALCITONI','TROPONIN','Pro_BNP','PTT','Vitamin_D','IL6']]
    # df_main_new2[ ['WBC','PNN','Lymphocytes','Hemoglobin','Platelets','Creatinine','ALT','LDH','FERRITIN','D_DIMER','CRP','PROCALCITONI','TROPONIN','Pro_BNP','PTT','IL6','Blood Group']].to_csv('prepare_association_df_Lab_'+df_name+'.csv')
    # return df_main_new2[ ['WBC','PNN','Lymphocytes','Hemoglobin','Platelets','Creatinine','ALT','LDH','FERRITIN','D_DIMER','CRP','PROCALCITONI','TROPONIN','Pro_BNP','PTT','IL6','Blood Group']]


# Clean features

In [225]:
draw_missing_data_table(main_lab_df)

,Total,Percent
MPA.Date of first prescription.2,309,0.993569
IS.Corticosteroids Date,307,0.987138
MPA.Date of first prescription.1,305,0.980707
C.Date,297,0.954984
C.Organ,294,0.945338
...,...,...
C.Hemodialysis,0,0.000000
C.End stage renal,0,0.000000
C.Cardiomyopathies,0,0.000000
C.Heart failure,0,0.000000


In [226]:
#Split BP column into two columns
main_lab_df["PEFF.BP"] = main_lab_df["PEFF.BP"].replace('%', '')
main_lab_df["PEFF.BP"] = main_lab_df["PEFF.BP"].replace(to_replace=r'\\', value='/', regex=True)
main_lab_df["PEFF.BP"] = main_lab_df["PEFF.BP"].replace(to_replace=r'Ø', value='', regex=True)
main_lab_df["PEFF.BP"] = main_lab_df["PEFF.BP"].replace(to_replace=r'%', value='', regex=True)
main_lab_df["PEFF.BP"] = main_lab_df["PEFF.BP"].replace(to_replace=r',', value='', regex=True)
main_lab_df["PEFF.BP"] = main_lab_df["PEFF.BP"].replace(to_replace=r'ظ', value='/', regex=True)
main_lab_df[['PEFF_BP_Systolic', 'PEFF_BP_Diastolic']] = main_lab_df['PEFF.BP'].str.split("[/|,]", expand = True)

In [227]:
# # all featrues
# features=['PatientId', 'AGE', 'G.Gender', 'G.Nationality','C_Diabetes', 'C_HTN', 'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies', 'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
#    'C.Lung Interstitial Disease ', 'C.Bronchial Asthma', 'C.Cerebrovascular', 'C.Neurologic (dementia)', 'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease', 'C.Obesity',
#    'C.Sick cell', 'C.Cancer','C.Solid organ transplant', 'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
#    'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies', 'IS.Pregnancy','IS.Smoker', 'ABGF_Ph',
#    'ABGF_Pa_O2', 'ABGF_Pa_CO2', 'ABGF_HCO3', 'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion', 'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia', 'CSA.Headache',
#    'CSA.Abdominal Pain', 'CSA.Nausea or Vomiting', 'CSA.Loss of Smell or Tast', 'PO.OUTCOME', 'PO.Condition', 'PO.Current Condition', 'PEFF_Temperature', 'PEFF_Respiratory_Rate',
#    'PEFF_Pulse', 'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow', 'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
#    'PEFF.If patient need prone position', 'PEFF.If patient intubated', 'PEFF.If Patient required Psychiatric Consultation', 'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
#    'PEFF.Confirmed Pulmonary embolism', 'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA','MPA_Antibiotics','MPA_Favipiravir',
#    'MPA.Kaletra +Ribavirin +Interferon','MPA.Hydroxychloroquine', 'MPA_Dexamethasone',
#    'MPA_Convalescent_plasma_transfusion','MPA_Clexan_or_Heparine',
#    'MPA_Tocilizumab', 'visits_y','WBC', 'PNN', 'Lymphocytes', 'Hemoglobin', 'Platelets', 'Creatinine', 'ALT', 'LDH', 'FERRITIN', 'D_DIMER', 'CRP', 'PROCALCITONI', 'TROPONIN', 'Pro_BNP',
#    'PTT', 'Vitamin_D', 'IL6', 'Blood Group', 'days_in_Hospital','Normal', 'Presence of consolidation', 'Presence of ground glass opacities', 'Bilateral or unilateral', 'Involve ≥ 50% the lung', 'Other']

In [228]:
# all featrues
features=['PatientId', 'AGE', 'Gender', 'G.Nationality','C_Diabetes', 'C_HTN', 'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies', 'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
   'C.Lung Interstitial Disease ', 'C.Bronchial Asthma', 'C.Cerebrovascular', 'C.Neurologic (dementia)', 'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease', 'C.Obesity',
   'C.Sick cell', 'C.Cancer','C.Solid organ transplant', 'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
   'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies', 'IS.Pregnancy','IS.Smoker', 'ABGF_Ph',
   'ABGF_Pa_O2', 'ABGF_Pa_CO2', 'ABGF_HCO3', 'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion', 'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia', 'CSA.Headache',
   'CSA.Abdominal Pain', 'CSA.Nausea or Vomiting', 'CSA.Loss of Smell or Tast', 'PO.OUTCOME', 'PO.Condition', 'PO.Current Condition', 'PEFF_Temperature', 'PEFF_Respiratory_Rate',
   'PEFF_Pulse', 'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow', 'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
   'PEFF.If patient need prone position', 'PEFF.If patient intubated', 'PEFF.If Patient required Psychiatric Consultation', 'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
   'PEFF.Confirmed Pulmonary embolism', 'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA',
   'MPA_Antibiotics','MPA_Favipiravir','MPA_Convalescent_plasma_transfusion','MPA_Tocilizumab',
   'visits_y','WBC', 'PNN', 'Lymphocytes', 'Hemoglobin', 'Platelets', 'Creatinine', 'ALT', 'LDH', 'FERRITIN', 'D_DIMER', 'CRP', 'PROCALCITONI', 'TROPONIN', 'Pro_BNP',
   'PTT', 'Vitamin_D', 'IL6', 'Blood Group', 'days_in_Hospital','XrayResult']

In [229]:
#featrues to process with jenks 
jenksfeatures=['AGE','ABGF_Ph','ABGF_Pa_O2', 'ABGF_Pa_CO2', 'ABGF_HCO3','PEFF_Temperature', 'PEFF_Respiratory_Rate','PEFF_Pulse', 'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow',
 'WBC', 'PNN', 'Lymphocytes', 'Hemoglobin', 'Platelets', 'Creatinine', 'ALT', 'LDH', 'FERRITIN', 'D_DIMER', 'CRP', 'PROCALCITONI', 'TROPONIN', 'Pro_BNP', 'PTT', 'Vitamin_D', 'IL6']

In [230]:
# main_lab_df.to_csv('toscv.csv')

In [231]:
main_lab_df[jenksfeatures]

,AGE,ABGF_Ph,ABGF_Pa_O2,ABGF_Pa_CO2,ABGF_HCO3,PEFF_Temperature,PEFF_Respiratory_Rate,PEFF_Pulse,PEFF_BP_Systolic,PEFF_BP_Diastolic,...,LDH,FERRITIN,D_DIMER,CRP,PROCALCITONI,TROPONIN,Pro_BNP,PTT,Vitamin_D,IL6
0,70.0,7.42,57.7,41.0,25.8,38.0,22.0,98,134,70,...,251,439,0.46,123,0.12,0.009,46,44.0,26,93.1
1,67.0,7.44,63.0,38.9,26.9,37.9,47.0,128,129,90,...,477,729,0.75,81,0.54,0.008,87,34.0,0,0
2,55.0,0.00,0.0,0.0,0.0,39.4,20.0,83,125,83,...,448,1384,0.45,33,0.11,FALSE,68,44.0,38,0
3,44.0,7.38,47.4,35.0,27.0,37.7,32.0,113,135,79,...,225,260,0.31,FALSE,0,FALSE,0,36.0,0,0
4,72.0,7.45,52.3,35.5,24.3,38.6,27.0,79,152,61,...,344,635,FALSE,FALSE,0.22,0.014,287,47.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,35.0,7.37,7.6 Kkpa,4.1 kpa,17.9,36.6,24,92,130,84,...,335,902,500,215,0.17,0.013,0,32.0,0,1.6
307,72.0,7.44,33,38,26.0,38.6,20,91,122,72,...,476,229,770,46,0.13,0,174,38.0,0,117
308,81.0,7.37,61,51,33.0,37.0,52,94,104,56,...,291,2767,4772,106,0.22,0.022,303,36.0,143,10.7
309,70.0,7.44,9.6 KPA,5.3 kpa,27.2,37.0,38,68,121,75,...,471,257,576,9,0.07,0.016,0,31.0,0,0


In [232]:
#features not process for jenks 
otherfeatures=['C_Diabetes', 'C_HTN', 'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies', 'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
   'C.Lung Interstitial Disease ', 'C.Bronchial Asthma', 'C.Cerebrovascular', 'C.Neurologic (dementia)', 'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease', 'C.Obesity',
   'C.Sick cell', 'C.Cancer','C.Solid organ transplant', 'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
   'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies', 'IS.Pregnancy','IS.Smoker', 
    'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion', 'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia', 'CSA.Headache',
   'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
   'PEFF.If patient need prone position', 'PEFF.If patient intubated', 'PEFF.If Patient required Psychiatric Consultation', 'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
   'PEFF.Confirmed Pulmonary embolism', 'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA', 'CSA.Nausea or Vomiting','CSA.Loss of Smell or Tast','PO.Condition',
   'PO.Current Condition','CSA.Abdominal Pain',
   'MPA_Antibiotics','MPA_Favipiravir','MPA_Convalescent_plasma_transfusion','MPA_Tocilizumab',
   'XrayResult'
   ] 

In [233]:
y=otherfeatures+jenksfeatures
list(set(main_lab_df.columns)-set(y))

['MPA.Hydroxychloroquine',
 'PEFF.Room Air Blood Oxygen Saturation',
 'IS.Corticosteroids Dose',
 'MPA.Date of first prescription.4',
 'G.Date of Death',
 'PEFF.Others.1',
 'visits_y',
 'ABGF.Arterial Blood Gazes',
 'IS.Corticosteroids Date',
 'MPA.Precise Dose',
 'G.Date of COVID-19 PCR',
 'Bilateral or unilateral',
 'G.Date of Intubation',
 'visits_x',
 'G.Date of Extubation',
 'Presence of consolidation',
 'Reason',
 'MPA.Precise',
 'MPA.Kaletra +Ribavirin +Interferon',
 'C.Cancer Details',
 'PEFF.BP',
 'MPA.Date of first prescription',
 'Presence of ground glass opacities',
 'G.Date of Symptoms Onset',
 'MPA.Date of first prescription.2',
 'IS.Pregnancy Weeks',
 'MPA_Dexamethasone',
 'MPA.Date of first prescription.1',
 'PO.OUTCOME',
 'C.Organ',
 'MPA_Clexan_or_Heparine',
 'G.Date of Admission',
 'MPA.Date of first prescription.5',
 'PEFF.Patient need Oxygen',
 'CSA.Others',
 'PEFF.Oxygen Liter(s)',
 'Blood Group',
 'Gender',
 'MPA.Date of first prescription.3',
 'MPA.Date of first

In [234]:
# #features not process for jenks 
# otherfeatures=['C_Diabetes', 'C_HTN', 'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies', 'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
#    'C.Lung Interstitial Disease ', 'C.Bronchial Asthma', 'C.Cerebrovascular', 'C.Neurologic (dementia)', 'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease', 'C.Obesity',
#    'C.Sick cell', 'C.Cancer','C.Solid organ transplant', 'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
#    'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies', 'IS.Pregnancy','IS.Smoker', 
#     'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion', 'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia', 'CSA.Headache',
#    'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
#    'PEFF.If patient need prone position', 'PEFF.If patient intubated', 'PEFF.If Patient required Psychiatric Consultation', 'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
#    'PEFF.Confirmed Pulmonary embolism', 'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA','MPA_Antibiotics','MPA_Favipiravir',
#    'MPA.Kaletra +Ribavirin +Interferon','MPA.Hydroxychloroquine', 'MPA_Dexamethasone',
#    'MPA_Convalescent_plasma_transfusion','MPA_Clexan_or_Heparine',
#    'MPA_Tocilizumab',  'Presence of consolidation', 'Presence of ground glass opacities'] 

In [235]:
#Deleate kpa word and only keep the number Example--> 50 kpa will be 50 
main_lab_df["ABGF_Pa_O2"] = main_lab_df["ABGF_Pa_O2"].str.split(' ').str[0]
main_lab_df["ABGF_Pa_O2"] = main_lab_df["ABGF_Pa_O2"].str.split(' ').str[0]
main_lab_df['ABGF_Pa_CO2'] = main_lab_df['ABGF_Pa_CO2'].str.split(' ').str[0]
main_lab_df['ABGF_Pa_CO2'] = main_lab_df['ABGF_Pa_CO2'].str.split(' ').str[0]

In [236]:
#replace , with . 
for feature in jenksfeatures:
    main_lab_df[feature ]=main_lab_df[feature].replace('0.0','FALSE') #Replace 0 with False
    main_lab_df[feature ]=main_lab_df[feature].replace(0.00,'FALSE') #Replace 0 with False
    main_lab_df[feature ]=main_lab_df[feature].replace('0','FALSE') #Replace 0 with False
    main_lab_df[feature ]=main_lab_df[feature].replace(to_replace =',', value ='.') 
    main_lab_df[feature] = main_lab_df[feature].replace(to_replace=r',', value='.', regex=True)
    main_lab_df[feature] = main_lab_df[feature].replace(to_replace=r'ز', value='', regex=True)
  


In [237]:
main_lab_df.shape

(311, 130)

In [238]:
#convert string to binary 
main_lab_df['IS.Pregnancy' ]=main_lab_df['IS.Pregnancy'].replace(to_replace ='FALSE', value =0)
main_lab_df['IS.Pregnancy']=main_lab_df['IS.Pregnancy'].replace(to_replace ='yes', value =1)

In [239]:
#convert string to binary 
cleanFeatures=['PEFF.Presence of thrombo-embolic','PEFF.If Patient required Psychiatric Consultation','PEFF.If patient intubated','PEFF.If patient need prone position',
               'MPA.Kaletra +Ribavirin +Interferon','MPA.Hydroxychloroquine', 'MPA_Dexamethasone','MPA_Convalescent_plasma_transfusion','MPA_Clexan_or_Heparine','MPA_Favipiravir',
               'MPA_Antibiotics','MPA_Tocilizumab']
for cleanFeature in cleanFeatures:
    main_lab_df[cleanFeature ]=main_lab_df[cleanFeature].replace(to_replace ='no', value =0)
    main_lab_df[cleanFeature]=main_lab_df[cleanFeature].replace(to_replace ='yes', value =1)


In [240]:
#PEFF_Pulse range - 60 to 100 beats per minute
main_lab_df = main_lab_df[~(main_lab_df["PEFF_Pulse"].isnull())]
main_lab_df["PEFF_Pulse"] = main_lab_df["PEFF_Pulse"].replace(to_replace=r'\\', value='/', regex=True)
main_lab_df["PEFF_Pulse"] = main_lab_df["PEFF_Pulse"].str.split('/').str[0]
main_lab_df["PEFF_Pulse"] = main_lab_df["PEFF_Pulse"].replace(to_replace=r'`', value='', regex=True)
main_lab_df['PEFF_Pulse'] = pd.to_numeric(main_lab_df['PEFF_Pulse'], errors='ignore')

In [241]:
#The Glasgow Coma Scale (GCS) is scored between 3 and 15, with 3 being the worst and 15 the best.
main_lab_df["PEFF_Glasgow"] = main_lab_df["PEFF_Glasgow"].replace(to_replace=r'\\', value='/', regex=True)
main_lab_df["PEFF_Glasgow"] = main_lab_df["PEFF_Glasgow"].str.split('/').str[0]
main_lab_df['PEFF_Glasgow'] = pd.to_numeric(main_lab_df['PEFF_Glasgow'], errors='ignore')

In [242]:
#calculate number of 0 values for all features
features_withZero=[]
for featureName in features:
     numberOfFalse=main_lab_df[featureName][(main_lab_df[featureName] == 0)].astype(int).count()
     total= main_lab_df[featureName].count()
     p=(50*total)/100 
     if (numberOfFalse>p):
         features_withZero.append(featureName)
        #  print(featureName, numberOfFalse,'Total: ', main_lab_df[featureName].count())

newFeatrue=[]
for n in otherfeatures:
    if n not in features_withZero:
        newFeatrue.append(n)     
added=['Gender']
otherfeatures=added+newFeatrue
print(otherfeatures)

['Gender', 'C_Diabetes', 'C_HTN', 'CSA_Fever', 'CSA_SOB', 'CSA_Cough', 'MPA_Antibiotics', 'MPA_Favipiravir', 'MPA_Tocilizumab', 'XrayResult']


In [243]:
#calculate number of False values for all features,  prent it if it's above 70%
features_withFasle=[]
for featureName in features:
     numberOfFalse=main_lab_df[featureName][(main_lab_df[featureName] == 'False') | (main_lab_df[featureName] == 'FALSE')].count()
     total= main_lab_df[featureName].count()
     p=(70*total)/100 
     if (numberOfFalse>p):
          features_withFasle.append(featureName)
          print(featureName, numberOfFalse,'Total: ', main_lab_df[featureName].count())

Vitamin_D 235 Total:  311


In [244]:
#Number of zero
for featureName in jenksfeatures:
     numberOfFalse=main_lab_df[featureName][(main_lab_df[featureName] == 0)].astype(int).count()
     total= main_lab_df[featureName].count()
     p=(50*total)/100 
     if (numberOfFalse>p):
        print(featureName, numberOfFalse,'Total: ', main_lab_df[featureName].count())

In [245]:
main_lab_df.shape

(311, 130)

In [246]:
#fix days 
main_lab_df['days_in_Hospital']=main_lab_df['days_in_Hospital'].astype('str')
main_lab_df['days_in_Hospital']=main_lab_df['days_in_Hospital'].str.replace('days','')
main_lab_df['days_in_Hospital']=main_lab_df['days_in_Hospital'].str.replace('NaT','0')
main_lab_df['days_in_Hospital']=main_lab_df['days_in_Hospital'].astype('int')

# Features Analysis

In [247]:
# df_main , df_lab,  df_xray

In [248]:
# all featrues
features=['PatientId', 'AGE', 'Gender', 'G.Nationality','C_Diabetes', 'C_HTN', 'C.Heart ischemic', 'C.Heart failure', 'C.Cardiomyopathies', 'C.End stage renal', 'C.Hemodialysis', 'C.COPD',
   'C.Lung Interstitial Disease ', 'C.Bronchial Asthma', 'C.Cerebrovascular', 'C.Neurologic (dementia)', 'C.History of psychiatric', 'C.Cirrhosis', 'C.liver disease', 'C.Obesity',
   'C.Sick cell', 'C.Cancer','C.Solid organ transplant', 'IS.Hematopoietic cell transplant', 'IS.HIV', 'IS.corticosteroids',
   'IS.Other immunosuppressing agents', 'IS.Other immunodeficiencies', 'IS.Pregnancy','IS.Smoker', 'ABGF_Ph',
   'ABGF_Pa_O2', 'ABGF_Pa_CO2', 'ABGF_HCO3', 'CSA_Fever', 'CSA_SOB', 'CSA.Chest pain', 'CSA.Confusion', 'CSA.Hemoptysis', 'CSA.Diarrhea', 'CSA_Cough', 'CSA.Myalgia', 'CSA.Headache',
   'CSA.Abdominal Pain', 'CSA.Nausea or Vomiting', 'CSA.Loss of Smell or Tast', 'PO.OUTCOME', 'PO.Condition', 'PO.Current Condition', 'PEFF_Temperature', 'PEFF_Respiratory_Rate',
   'PEFF_Pulse', 'PEFF_BP_Systolic', 'PEFF_BP_Diastolic', 'PEFF_Glasgow', 'PEFF.Nasal Cannula', 'PEFF.Mask', 'PEFF.HFNC',
   'PEFF.If patient need prone position', 'PEFF.If patient intubated', 'PEFF.If Patient required Psychiatric Consultation', 'PEFF.Presence of thrombo-embolic', 'PEFF.Confirmed DVT',
   'PEFF.Confirmed Pulmonary embolism', 'PEFF.Confirmed Myocardial infarction', 'PEFF.Confirmed CVA/TIA',
   'MPA_Antibiotics','Anticoagulant','Immunomodulators','antiviral',
   'visits_y','WBC', 'PNN', 'Lymphocytes', 'Hemoglobin', 'Platelets', 'Creatinine', 'ALT', 'LDH', 'FERRITIN', 'D_DIMER', 'CRP', 'PROCALCITONI', 'TROPONIN', 'Pro_BNP',
   'PTT', 'Vitamin_D', 'IL6', 'Blood Group', 'days_in_Hospital','XrayResult']

In [249]:
#features not process for jenks 
otherfeatures=['Gender', 'C_Diabetes', 'C_HTN', 'CSA_Fever', 'CSA_SOB', 'CSA_Cough', 'MPA_Antibiotics','Anticoagulant','Immunomodulators','antiviral', 'XrayResult']


In [250]:
main_lab_df=pd.read_csv(path+'discharge_final_ML.csv')

In [251]:
for feature in ['Gender']: 
    main_lab_df[feature] = main_lab_df[feature].map({'male':1,'female':0})

In [252]:
main_lab_df.shape

(248, 93)

In [253]:
target = main_lab_df['OUTCOME']
print(f"train data class counts: {Counter(target)}")

train data class counts: Counter({'discharge=<1': 85, 'discharge=<2': 80, 'discharge=<3': 37, 'discharge>4': 34, 'discharge=<4': 12})


In [254]:
df_name='discharge=<4'#'discharge=<1' ' discharge=<2' ' discharge=<3' ' discharge>4' ' discharge=<4'
support=0.5
confidence=0.7
min_len=3
max_len = 10

In [255]:
main_lab_df[otherfeatures]

,Gender,C_Diabetes,C_HTN,CSA_Fever,CSA_SOB,CSA_Cough,MPA_Antibiotics,Anticoagulant,Immunomodulators,antiviral,XrayResult
0,1,1,1,0,1,0,1,0,0,0,1
1,0,0,0,0,0,0,1,1,0,0,1
2,0,1,0,0,0,1,1,0,0,0,1
3,1,1,1,1,1,1,1,1,0,0,1
4,1,1,1,1,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
243,0,0,0,0,1,0,1,1,1,1,1
244,1,0,1,1,1,0,1,1,1,1,1
245,1,1,1,0,1,1,1,1,1,1,1
246,0,1,1,1,1,1,1,1,1,1,1


In [256]:
#use for association mining (spisific the Visits and the Outcome)
main_lab_df=main_lab_df[(main_lab_df['OUTCOME']==df_name)] #died ,discharge

In [257]:
# Run Jenks on spisific visits
main_lab_Jenks_df=main_lab_df#[(main_lab_df['visits_y'].astype(int)==1)]

In [258]:
#SAva the Dataset
# main_lab_df[features].to_csv(path+'df_main_lab_final_ML.csv',index=False)

In [259]:
# # set the days range
# main_lab_df=main_lab_df[(main_lab_df['days_in_Hospital'].between(0,7))]

In [260]:
main_lab_df.shape

(12, 93)

In [261]:
#Replace False with 600000 in order to use it with jenkes
df_main_new_copy=main_lab_df.copy(deep=True)
df_main_new_copy=df_main_new_copy.replace(to_replace ='FALSE', value =600000)
df_main_new_copy=df_main_new_copy.replace(to_replace ='False', value =600000)
for featureName in jenksfeatures: 
    df_main_new_copy[featureName]= pd.to_numeric(df_main_new_copy[featureName], downcast="float")


In [262]:
main_lab_Jenks_df.shape

(12, 93)

In [263]:
# df_main_new=main_lab_df.copy(deep=True)
# count=1
# # avoid Features that have insufficient Data #,'CPK','Vitamin_D','Fibrinogen','AST'
# # avoidFeatureList=['INR','CPK','Vitamin_D','Fibrinogen','AST']
# avoidFeatureList=['INR','AST','CPK','Fibrinogen']#,
# for featureName in jenksfeatures:
#     #Avoid INR future
#     if featureName in avoidFeatureList: continue

#     #Avoid False values
#     #main_lab_df = main_lab_df[~(main_lab_df[featureName] == 'False') & ~(main_lab_df[featureName] == 'FALSE')]
#     main_lab_Jenks_df = main_lab_Jenks_df[~(main_lab_Jenks_df[featureName] == 'False') & ~(main_lab_Jenks_df[featureName] == 'FALSE')]

#     #Convet  to numeric
#    # main_lab_df[featureName] = pd.to_numeric(main_lab_df[featureName], errors='ignore')
#     df_main_new[featureName] = pd.to_numeric(df_main_new[featureName], errors='ignore')
#     main_lab_Jenks_df[featureName] = pd.to_numeric(main_lab_Jenks_df[featureName], errors='ignore')


#     # use JenksNaturalBreaks 
#     jnb = JenksNaturalBreaks()
#     featureVlaues=main_lab_Jenks_df[featureName]
#     featureVlaues.to_list()
#     jnb.fit(featureVlaues)

#     try:
#         # print(jnb.labels_)
#         # print(jnb.groups_)
#         jenksOutput=getJenks(main_lab_Jenks_df,featureName,jnb.inner_breaks_)
#         jenksOutput.to_csv('output/jenksOutput_Visit1.csv', mode='a', header=True)
        # newdata=convertValues(df_main_new,df_main_new_copy,featureName,jnb.inner_breaks_)

#     except:
#         pass
#         print('error')


In [264]:
#The New Jenks based on Domain expert feedback 'AGE':[0,18,50,65],
newJenks={
    
    'AGE':[0,37,55,73],
    'ABGF_Ph':[0,7.35,7.45],
    'ABGF_Pa_O2':[0,80],
    'ABGF_Pa_CO2':[0,35,45],
    'ABGF_HCO3':[0,21,27],
    'PEFF_Temperature':[0,36,37.6,38.6],
    'PEFF_Respiratory_Rate':[12,20,28],
    'PEFF_Pulse':[79.0,95.0,111.0,134.0,185.0],
    'PEFF_BP_Systolic':[0,90,130],
    'PEFF_BP_Diastolic':[0,60,90],
    'PEFF_Glasgow':[3,4,8,12,15],
    'WBC':[0,4000,11000],
    'PNN':[500,1000,7700,15000,],
    'Lymphocytes':[1000,4000],
    'Hemoglobin':[8,10,12],
    'Platelets':[50000,150000,450000],
    'Creatinine':[59,104,250,500],
    'ALT':[0,41],
    'LDH':[135,225],
    'FERRITIN':[792.0,1976.0,4374.0,7627.0,159000.0],
    'D_DIMER':[0,500],
    'CRP':[0,6,100],
    'PROCALCITONI':[0.25,0.5],
    'TROPONIN':[0,0.1],
    'Pro_BNP':[0,125,450],
    'PTT':[0,11.5,14.5],
    'Vitamin_D':[50,250],
    'IL6':[0,37.5]
                }

In [265]:
df_main_new=main_lab_df.copy(deep=True)
count=1
# avoid Features that have insufficient Data #,'CPK','Vitamin_D','Fibrinogen','AST'
# avoidFeatureList=['INR','CPK','Vitamin_D','Fibrinogen','AST']
avoidFeatureList=['INR','AST','CPK','Fibrinogen']#,
for featureName in jenksfeatures:
    #Avoid INR future
    if featureName in avoidFeatureList: continue

    #Avoid False values
    #main_lab_df = main_lab_df[~(main_lab_df[featureName] == 'False') & ~(main_lab_df[featureName] == 'FALSE')]
    main_lab_Jenks_df = main_lab_Jenks_df[~(main_lab_Jenks_df[featureName] == 'False') & ~(main_lab_Jenks_df[featureName] == 'FALSE')]

    #Convet  to numeric
   # main_lab_df[featureName] = pd.to_numeric(main_lab_df[featureName], errors='ignore')
    df_main_new[featureName] = pd.to_numeric(df_main_new[featureName], errors='ignore')
    main_lab_Jenks_df[featureName] = pd.to_numeric(main_lab_Jenks_df[featureName], errors='ignore')

try:
    for feature,breakPoints in newJenks.items():
        newdata=convertValues(df_main_new,df_main_new_copy,feature,breakPoints)
except:
    pass
    print('error')


In [266]:
main_lab_Jenks_df.shape

(1, 93)

In [267]:
df_main_other=df_main_new[otherfeatures]


In [268]:
for featureName in otherfeatures:
    df_main_other.loc[df_main_other[featureName] == 1, featureName] = featureName+':1'
    df_main_other.loc[df_main_other[featureName] == 0, featureName] = featureName+':0'
    df_main_other.loc[df_main_other[featureName] == '1', featureName] = featureName+':1'
    df_main_other.loc[df_main_other[featureName] == '0', featureName] = featureName+':0'
    df_main_other[[featureName]]=df_main_other[[featureName]].replace('FALSE', featureName+':False')

/tmp/ipykernel_661762/3541962094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main_other.loc[df_main_other[featureName] == 1, featureName] = featureName+':1'
/tmp/ipykernel_661762/3541962094.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main_other[[featureName]]=df_main_other[[featureName]].replace('FALSE', featureName+':False')
/tmp/ipykernel_661762/3541962094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [269]:
df_main_other.shape

(12, 11)

In [270]:
df_main_new.shape

(12, 93)

In [271]:
# newdata=newdata.replace(to_replace =600000, value ='False600000')


In [272]:
for featureName in jenksfeatures:

    newdata[[featureName]]=newdata[[featureName]].replace(600000, featureName+':False')

/tmp/ipykernel_661762/1989208729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata[[featureName]]=newdata[[featureName]].replace(600000, featureName+':False')
/tmp/ipykernel_661762/1989208729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata[[featureName]]=newdata[[featureName]].replace(600000, featureName+':False')
/tmp/ipykernel_661762/1989208729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [273]:
newdata.shape

(12, 28)

In [274]:
# newdata.to_csv('newdata.csv')

In [275]:
for featureName in jenksfeatures:
    newdata.loc[newdata[featureName] == 1, featureName] = featureName+':1'
    newdata.loc[newdata[featureName] == 0, featureName] = featureName+':0'

In [276]:
newdata.shape

(12, 28)

In [277]:
#Connect the dataset together
newdata= pd.concat([newdata, df_main_other], axis=1)

In [278]:
newdata.shape

(12, 39)

In [279]:
# newdata.to_csv(path+'prepare_association_df_Lab_'+df_name+'_'+str(support)+'_'+str(confidence)+'.csv')


In [280]:
#Deleted Columns
y=df_main_new.columns
list(set(y)-set(newdata.columns))

['CSA.Confusion',
 'PO.Condition',
 'CSA.Nausea or Vomiting',
 'C.Solid organ transplant',
 'IS.HIV',
 'C.COPD',
 'C.Cirrhosis',
 'C.Lung Interstitial Disease ',
 'PEFF.Presence of thrombo-embolic',
 'IS.Smoker',
 'AST',
 'PEFF.Confirmed CVA/TIA',
 'Fibrinogen',
 'CSA.Hemoptysis',
 'C.Heart ischemic',
 'C.Heart failure',
 'C.Neurologic (dementia)',
 'C.History of psychiatric',
 'CSA.Headache',
 'PEFF.HFNC',
 'PEFF.Mask',
 'PEFF.If patient need prone position',
 'PEFF.If Patient required Psychiatric Consultation',
 'PEFF.Confirmed DVT',
 'IS.Pregnancy',
 'IS.corticosteroids',
 'IS.Other immunosuppressing agents',
 'C.Cancer',
 'C.Sick cell',
 'CSA.Loss of Smell or Tast',
 'CSA.Diarrhea',
 'PEFF.Nasal Cannula',
 'C.liver disease',
 'C.Cardiomyopathies',
 'C.Cerebrovascular',
 'CSA.Chest pain',
 'CPK',
 'IS.Other immunodeficiencies',
 'PEFF.If patient intubated',
 'PEFF.Confirmed Myocardial infarction',
 'IS.Hematopoietic cell transplant',
 'OUTCOME',
 'C.Bronchial Asthma',
 'C.End stage 

In [281]:
shape=newdata.shape
column=shape[0]
row=shape[1]
print(column,row)

12 39


In [282]:
#Converting the pandas dataframe into a list of lists
records = []
for i in range(column):#number of Row
    records.append ([str(newdata.values[i,j]) for j in range(row)]) #number of column

In [283]:
#Building the first priori model
#min_confidence 60 , 70 
association_rules=apriori(records, min_support=support, min_confidence=confidence, min_lift=0.5, min_length=min_len, max_length = max_len)
association_results = list (association_rules)

In [284]:
# #Glancing at the first rule
# df_t=pd.DataFrame(association_results)
# df_t.head()
# df_t.to_csv('test_'+df_name+'_visit1.csv')

In [285]:
#filter the result
association_results = filter(lambda x: len(x.items) > 2, association_results)

In [286]:
output = list(association_results) # returns a non-tabular output
with open(path+'CSV_Association_results_Lab_'+df_name+'_'+str(support)+'_'+str(confidence)+'_visit1.csv','w') as writeFile:
    csvWrite=csv.writer(writeFile,delimiter=',',quoting=csv.QUOTE_MINIMAL)
    csvWrite.writerow(['Rule','Support','Confidence','Lift'])
    rule=''
    for item in output:
       
        # first index of the inner list
        # Contains base item and add item
        pair = item[0] 
        items = [x for x in pair]
        items.sort()
        if(len(items)>=2):
            rule= str(items)
            print(rule)
            # writeFile.writelines("Rule: " + str(items)+ '\n')
        # else:
        #     rule= str(items)
            # writeFile.writelines("Rule: " + str(items)+ '\n')
        #second index of the inner list
        # writeFile.writelines("Support: " + str(item[1])+ '\n')
        supportRule= str(item[1])
        #third index of the list located at 0th
        #of the third index of the inner list
        # writeFile.writelines("Confidence: " + str(item[2][0][2])+ '\n')
        confidenceRule=str(item[2][0][2])
        # writeFile.writelines("Lift: " + str(item[2][0][3])+ '\n')
        liftRule=str(item[2][0][3])
        # writeFile.writelines("====================================="+ '\n')
        csvWrite.writerow([rule,supportRule,confidenceRule,liftRule])
         

['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'Anticoagulant:1']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'C_HTN:1']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'Immunomodulators:1']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'LDH:> :225']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'MPA_Antibiotics:1']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'PEFF_Glasgow:12_15']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'PNN:1000_7700']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'PTT:> :14.5']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'Platelets:< :50000']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'WBC:4000_11000']
['ABGF_HCO3:21_27', 'ABGF_Pa_O2:0_80', 'antiviral:1']
['ABGF_HCO3:21_27', 'Anticoagulant:1', 'C_HTN:1']
['ABGF_HCO3:21_27', 'Anticoagulant:1', 'Immunomodulators:1']
['ABGF_HCO3:21_27', 'Anticoagulant:1', 'LDH:> :225']
['ABGF_HCO3:21_27', 'Anticoagulant:1', 'MPA_Antibiotics:1']
['ABGF_HCO3:21_27', 'Anticoagulant:1', 'PEFF_Glasgow:12_15']
['ABGF_HCO3:21_27', 'Anticoagulant:1', 'PNN:1000_7700']
['ABGF_HCO3:21_27', 'Anticoagulant

In [287]:
# #Getting the number of rules
# print(len(association_results))

In [288]:
#Glancing at the first rule
df=pd.DataFrame(output)
df.head()


,items,support,ordered_statistics
0,"(ABGF_HCO3:21_27, ABGF_Pa_O2:0_80, Anticoagula...",0.583333,"[((ABGF_HCO3:21_27), (ABGF_Pa_O2:0_80, Anticoa..."
1,"(ABGF_HCO3:21_27, C_HTN:1, ABGF_Pa_O2:0_80)",0.500000,"[((ABGF_HCO3:21_27), (ABGF_Pa_O2:0_80, C_HTN:1..."
2,"(ABGF_HCO3:21_27, Immunomodulators:1, ABGF_Pa_...",0.583333,"[((ABGF_HCO3:21_27), (Immunomodulators:1, ABGF..."
3,"(ABGF_HCO3:21_27, ABGF_Pa_O2:0_80, LDH:> :225)",0.500000,"[((ABGF_HCO3:21_27), (ABGF_Pa_O2:0_80, LDH:> :..."
4,"(ABGF_HCO3:21_27, MPA_Antibiotics:1, ABGF_Pa_O...",0.583333,"[((ABGF_HCO3:21_27), (ABGF_Pa_O2:0_80, MPA_Ant..."


In [289]:
# df.to_csv(path+'association_results_Lab_'+df_name+'_'+str(support)+'_'+str(confidence)+'_visit1.csv')


In [290]:
# output = list(association_results) # returns a non-tabular output
# print(output[0])
# for result in output:
#    x=(tuple(result[2][0][1]))
#    print(x)
#    print(len(x))
   
# # putting output into a pandas dataframe ('AST_Range(0.0, 0.0)',)
# lhs=[]
# def inspect(output):
#     for result in output:
#         if (len(result)>=2):
#             lhs.append([tuple(result[2][1])])
#         else:
#             lhs.append([tuple(result[2][0])[0]])
#     rhs         = [tuple(result[2][0][1])[0] for result in output]
#     support    = [result[1] for result in output]
#     confidence = [result[2][0][2] for result in output]
#     lift       = [result[2][0][3] for result in output]
#     return list(zip(lhs, rhs, support, confidence, lift))
# output_DataFrame = pd.DataFrame(inspect(output), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
# #(OrderedStatistic(items_base=frozenset(), items_add=frozenset({'AST_Range(0.0, 0.0)'}), confidence=0.9, lift=1.0),)
